In [3]:
import os
os.chdir("C:/.Upwork Projects/2025-06-11 Fin Adviser MVP/PythonJSONGenerator")

In [8]:
import sys
print(sys.executable)

C:\Users\pgrab\anaconda3\python.exe


In [5]:
# Uncomment and run if you haven't already installed these
!pip install faker pandas numpy



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import json
import random
import string
from datetime import datetime, timedelta
from faker import Faker
import numpy as np
import pandas as pd

fake = Faker()
random.seed(42)


In [2]:
NUM_COMPANIES = 100
OUTPUT_DIR = "output"
DETAILS_DIR = os.path.join(OUTPUT_DIR, "details")
os.makedirs(DETAILS_DIR, exist_ok=True)

INDUSTRIES = [
    "Technology", "Finance", "Healthcare", "Retail", "Energy", "Telecommunications",
    "Real Estate", "Transportation", "Consumer Goods", "Utilities"
]


In [8]:
def random_ticker():
    return ''.join(random.choices(string.ascii_uppercase, k=random.randint(3, 5)))

def generate_company_summary():
    name = fake.company()
    ticker = random_ticker()
    industry = random.choice(INDUSTRIES)
    market_cap = f"{round(random.uniform(10, 2000), 1)}"
    current_price = round(random.uniform(10, 500), 2)
    intrinsic_min = round(current_price * random.uniform(0.9, 1.0), 2)
    intrinsic_max = round(current_price * random.uniform(1.0, 1.1), 2)
    return {
        "companyName": name,
        "ticker": ticker,
        "marketCap": market_cap,
        "industry": industry,
        "currentPrice": current_price,
        "intrinsicPrice": {
            "min": intrinsic_min,
            "max": intrinsic_max
        }
    }


In [9]:
def generate_stock_prices(current_price):
    base = current_price
    dates = [datetime.today() - timedelta(days=i) for i in range(365)]
    dates.reverse()
    prices = []
    for date in dates:
        base += random.uniform(-1.5, 1.5)
        base = max(5, base)
        prices.append({
            "date": date.strftime("%Y-%m-%d"),
            "price": round(base + random.uniform(-0.5, 0.5), 2)
        })
    return prices


In [23]:
def generate_intrinsic_estimates(prices):
    quarters = [
        ("Q1", "2024-01-01", "2024-04-01"),
        ("Q2", "2024-04-01", "2024-07-01"),
        ("Q3", "2024-07-01", "2024-10-01"),
        ("Q4", "2024-10-01", "2025-01-01"),
          ("Q5", "2025-01-01", "2025-04-01"),
        ("Q6", "2025-04-01", "2025-06-30"),
    ]
    estimates = []
    for q, start, end in quarters:
        avg_price = np.mean([p['price'] for p in prices])
        min_val = round(avg_price * random.uniform(0.95, 1.0), 2)
        max_val = round(avg_price * random.uniform(1.0, 1.05), 2)
        estimates.append({
            "period": q,
            "startDate": start,
            "endDate": end,
            "min": min_val,
            "max": max_val
        })
    return estimates


In [24]:
def generate_evaluation_metrics():
    metrics_config = {
        "P/E": {
            "description": "Price-to-Earnings Ratio",
            "range": (5, 40),
            "label_fn": lambda v: "good" if v < 15 else "neutral" if v < 25 else "bad"
        },
        "ROE": {
            "description": "Return on Equity",
            "range": (0.02, 0.25),
            "label_fn": lambda v: "good" if v > 0.15 else "neutral" if v > 0.08 else "bad"
        },
        "Debt/Equity": {
            "description": "Debt-to-Equity Ratio",
            "range": (0.1, 3),
            "label_fn": lambda v: "good" if v < 1 else "neutral" if v < 2 else "bad"
        },
        "Current Ratio": {
            "description": "Current Assets / Current Liabilities",
            "range": (0.8, 3),
            "label_fn": lambda v: "good" if v > 1.5 else "neutral" if v > 1 else "bad"
        },
        "Margin": {
            "description": "Net Profit Margin",
            "range": (0.05, 0.3),
            "label_fn": lambda v: "good" if v > 0.2 else "neutral" if v > 0.1 else "bad"
        },
        "Beta": {
            "description": "Volatility Compared to Market",
            "range": (0.6, 1.8),
            "label_fn": lambda v: "good" if v < 1 else "neutral" if v < 1.3 else "bad"
        },
        "PBV": {
            "description": "Price-to-Book Value",
            "range": (0.5, 8),
            "label_fn": lambda v: "good" if v < 3 else "neutral" if v < 5 else "bad"
        },
        "Dividend Yield": {
            "description": "Annual Dividends / Share Price",
            "range": (0.0, 0.06),
            "label_fn": lambda v: "good" if v > 0.03 else "neutral" if v > 0.01 else "bad"
        }
    }

    years = [2022, 2023, 2024]
    metrics_list = []

    for name, config in metrics_config.items():
        values = []
        for year in years:
            value = round(random.uniform(*config["range"]), 3)
            label = config["label_fn"](value)
            values.append({
                "year": year,
                "value": value,
                "label": label
            })
        metrics_list.append({
            "name": name,
            "description": config["description"],
            "values": values
        })

    return metrics_list


In [25]:
company_summaries = []

for _ in range(NUM_COMPANIES):
    summary = generate_company_summary()
    company_summaries.append(summary)

    daily_prices = generate_stock_prices(summary["currentPrice"])
    
    details = {
        "companyName": summary["companyName"],
        "description": fake.text(max_nb_chars=300),
        "dailyStockPrice": daily_prices,
        "intrinsicValueEstimates": generate_intrinsic_estimates(daily_prices),
        "evaluationMetrics": generate_evaluation_metrics()
    }

    ticker_file = os.path.join(DETAILS_DIR, f"{summary['ticker'].lower()}.json")
    with open(ticker_file, "w") as f:
        json.dump(details, f, indent=2)

with open(os.path.join(OUTPUT_DIR, "companies-list.json"), "w") as f:
    json.dump(company_summaries, f, indent=2)

print("✅ Done! Files are saved in the 'output' folder.")


✅ Done! Files are saved in the 'output' folder.
